In [2]:
import os
import win32com.client as win32
import pandas as pd

In [ ]:
pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)

# Misc

In [ ]:
rest_of_email = '''
Please see attached for our summer installment of Knightsbridge’s quarterly market commentary and your quarterly report.\n
In this letter we discuss:\n
• The wild advance of big tech stocks in an apparent AI boom…. or bubble
• How this makes the index so concentrated in a few names that it is no longer representative of the “average” stock
• How that can be a concerning sign of a potential unwind
• Our high-level thoughts on the AI ecosystem
• That housing looks expensive as well
• Why we continue to avoid allocations to private equity
• How this is all occurring against the backdrop of a weakening economy\n
Thank you,
'''

# Define the full path of the subfolder
subfolder_path = os.path.join(os.getcwd(), 'pdf-reports')

# Read the Excel file
df = pd.read_excel(r'C:\Users\MarcoHui\Desktop\testing_emails.xlsx')

# Create an instance of Outlook
outlook = win32.Dispatch('outlook.application')

# Loop through the rows in the dataframe
for index, row in df.iterrows():
    email_1 = row['email_1']
    email_2 = row['email_2']
    filepath_1 = row['filepath_1']
    filepath_2 = row['filepath_2']
    client_name = row['name']

    # Create the full paths for the file attachments
    full_filepath_1 = os.path.join(subfolder_path, filepath_1)
    full_filepath_2 = os.path.join(subfolder_path, filepath_2)

    # Create a new email
    mail = outlook.CreateItem(0)  # 0: olMailItem
    mail.To = email_1

    if pd.notna(email_2):  # Add the second email if it is not empty
        mail.To += f';{email_2}'

    mail.Subject = 'Knightsbridge Quarterly Market Commentary and Report'
    mail.Body = f'Dear {client_name},\n{rest_of_email}'
    
    # Attach the file
    if os.path.exists(full_filepath_1) and os.path.exists(full_filepath_2) :
        mail.Attachments.Add(full_filepath_1)
        mail.Attachments.Add(full_filepath_2)

    # Save the email as a draft
    mail.Save()

    print(f'{client_name} is done!')

print("Draft emails created successfully!")

# Experimenting

### Reading file names

In [129]:
# Function to get a list of all filenames of a given folder
def get_filenames(folder,file_type):
    # Get all files in the folder
    all_files = os.listdir(folder)

    # Filter for specified file type
    pdf_files = [f for f in all_files if f.endswith(file_type)]

    # Create a DataFrame from the list of files
    df = pd.DataFrame(pdf_files, columns=['filename'])

    # Write the DataFrame to an Excel file
    #output_filepath = os.path.join(folder, 'pdf_name_list.xlsx')
    #df.to_excel(output_filepath, index=False)
    #print(f"Excel file saved at: {output_excel}")

    return df

In [130]:
reports_folderpath = r'C:\Users\MarcoHui\Knightsbridge Asset Management, LLC\Knightsbridge - Documents\Client Service\Quarterly Reports\2024\Q1\Team Kurt Final\Email'
reports_df = get_filenames(reports_folderpath, '.pdf')

In [131]:
# Remove filename suffix to get portfolio
reports_df['portfolio'] = reports_df['filename'].str.split().str[:-1].str.join(' ')

In [132]:
reports_df.head(3)

,filename,portfolio
0,A.J. Vater & Company 2024-03-31.pdf,A.J. Vater & Company
1,Alan and Lynne Goldin 2024-03-31.pdf,Alan and Lynne Goldin
2,Albert & Mackenzie LLP Portfolio (Copy Elizabe...,Albert & Mackenzie LLP Portfolio (Copy Elizabe...


### Reading contact emails

In [122]:
contacts_filepath = r'C:\Users\MarcoHui\Downloads\All Active Contacts 8-5-2024 9-58-27 AM.xlsx'
contacts_df = pd.read_excel(contacts_filepath)

c:\Users\MarcoHui\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [123]:
# Rename columns
contacts_df.columns = ['cpu_contact', 'cpu_checksum', 'cpu_modified', 
                       'full_name', 'first_name', 'middle_name', 'last_name', 'household', 'primary_advisor', 
                       'email', 'home_phone', 'mobile_phone', 'business_phone']

In [124]:
# Filter out rows without email addresses
contacts_df = contacts_df.dropna(subset=['email'])

In [125]:
contacts_df.head(3)

,cpu_contact,cpu_checksum,cpu_modified,full_name,first_name,middle_name,last_name,household,primary_advisor,email,home_phone,mobile_phone,business_phone
33,a0288d15-c1ae-4b3b-9f8d-762eb7859203,YbT8kemJiKm0k28dBMKvqmC7MtrIuSjY13WS8dtnViVjs2...,2024-05-05 06:03:00,11-99 Foundation,NaN,NaN,11-99 Foundation,11-99 Foundation,Knightsbridge Team,chp1199@aol.com,NaN,NaN,714-529-1199
94,338732e7-d171-4e87-a04d-cb2c15b0975b,RsaRNzlDyJL4gxPSEN7SEhjDCZwknCpt1lkionuvHTrO2T...,2024-05-05 06:11:38,"Abbott, Alec",Alec,NaN,Abbott,"Abbott, Alec",Knightsbridge Team,aabbott@squarmilner.com,NaN,NaN,(949) 222-2999
95,fa129a3e-30f0-4442-8cfb-a970d9092696,LZC0HHFT3CTNhLt6snG0D1/rCAVYq61/23cb4kALa5hU2Y...,2024-05-05 06:08:59,"Abbott, Brandon",Brandon,NaN,Abbott,"Abbott, Brandon","Prichard, John",brandon.l.abbott@gmail.com,NaN,(760) 419-6969,NaN


In [126]:
# Number of emails per household
contacts_df.groupby('household').agg(email_count=('email', 'count')).reset_index().sort_values(by='email_count', ascending=False)

,household,email_count
405,"Equity Advisor Solutions, Equity Advisor Solut...",5
424,"Fidelity Institutional Wealth Services, Fideli...",5
1024,"One Ford Road - Front gate, One Ford Road",5
13,"Advisors, Jeff",5
645,"Janus - Jordan Schroeder, Janus - Jordan Schro...",4
...,...,...
540,"Hanks, Pearl",1
539,"Hankin, Marc",1
538,"Hamontree, Mike & Joelle",1
537,"Hamilton, Rick & Peggy",1


In [127]:
# Create a helper column for the position of each value in column 'B'
contacts_df['email_position'] = contacts_df.groupby('household').cumcount() + 1

# Pivot the DataFrame
contacts_nodup_df = contacts_df.pivot_table(index='household', columns='email_position', values='email', aggfunc='first').reset_index()

# Rename columns
contacts_nodup_df.columns.name = None
contacts_nodup_df.columns = ['household'] + [f'email_{i}' for i in contacts_nodup_df.columns[1:]]

In [128]:
len(contacts_nodup_df)

1516

### Reading portfolio household key

In [133]:
key_filepath = r'C:\Users\MarcoHui\Downloads\groupname_household_key.xlsx'
key_df = pd.read_excel(key_filepath, sheet_name='key')

# Drop extra columns
key_df.drop(columns=['combination'], inplace=True)

In [134]:
key_df.head(5)

,portfolio,household
0,A.J. Vater & Company,"Vater, Daniel & Andrew"
1,Adrienne Cord,"Cord, Adrienne Hull"
2,Aedrean Kae Ye,"Ye, Aedrean"
3,Alan and Lynne Goldin,"Goldin, Alan & Lynne"
4,Alan and Sue Anne Cusey Portfolio,"Cusey, Alan and Sue Anne"


### Combining everything

In [155]:
joined_df = pd.merge(reports_df, key_df, on='portfolio', how='left')

In [156]:
# Create a new column that is True if the value in 'portfolio' is a duplicate
joined_df['duplicate'] = joined_df['portfolio'].duplicated(keep=False)

In [159]:
joined_df[joined_df['duplicate']==True]

,filename,portfolio,household,duplicate
20,Custom European Motor Cars 2024-03-31.pdf,Custom European Motor Cars,"Slaby, Tim, Sloan, Jason",True
21,Custom European Motor Cars 2024-03-31.pdf,Custom European Motor Cars,"Slaby, Tim, Smith, Steve",True
22,Custom European Motor Cars 2024-03-31.pdf,Custom European Motor Cars,"Slaby, Tim",True


In [161]:
joined_nodup_df = joined_df.drop_duplicates(subset=['portfolio'], keep='first')

In [164]:
joined_nodup_df[joined_nodup_df['duplicate']==True]

,filename,portfolio,household,duplicate
20,Custom European Motor Cars 2024-03-31.pdf,Custom European Motor Cars,"Slaby, Tim, Sloan, Jason",True


In [169]:
final_df = pd.merge(joined_nodup_df, contacts_nodup_df, on='household', how='left')

In [170]:
final_df.columns 

,filename,portfolio,household,duplicate,email_1.0,email_2.0,email_3.0,email_4.0,email_5.0
0,A.J. Vater & Company 2024-03-31.pdf,A.J. Vater & Company,"Vater, Daniel & Andrew",False,andy@ajvater.com,dan@ajvater.com,NaN,NaN,NaN
1,Alan and Lynne Goldin 2024-03-31.pdf,Alan and Lynne Goldin,"Goldin, Alan & Lynne",False,alanfgoldin@gmail.com,NaN,NaN,NaN,NaN
2,Albert & Mackenzie LLP Portfolio (Copy Elizabe...,Albert & Mackenzie LLP Portfolio (Copy Elizabe...,NaN,False,NaN,NaN,NaN,NaN,NaN
3,Albert & Mackenzie LLP Portfolio 2024-03-31.pdf,Albert & Mackenzie LLP Portfolio,"Albert, Bruce & Cindy",False,balbert@albmac.com,cindo411@gmail.com,NaN,NaN,NaN
4,Albert Family Portfolio 2024-03-31.pdf,Albert Family Portfolio,"Albert, Bruce & Cindy",False,balbert@albmac.com,cindo411@gmail.com,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
68,Sy Baker Portfolio 2024-03-31.pdf,Sy Baker Portfolio,"Baker, Sayoko",False,mssybaker@yahoo.com,NaN,NaN,NaN,NaN
69,TJ Leason 2024-03-31.pdf,TJ Leason,"Leason, TJ",False,tj@leasons.com,NaN,NaN,NaN,NaN
70,Uzayr Jeenah 2024-03-31.pdf,Uzayr Jeenah,"Jeenah, Uzayr",False,NaN,NaN,NaN,NaN,NaN
71,Victoria Kennedy Reel Portfolio 2024-03-31.pdf,Victoria Kennedy Reel Portfolio,"Reel, Victoria",False,NaN,NaN,NaN,NaN,NaN


### Creating GUI elements

In [ ]:
import tkinter as tk
from func_gui_elements import ask_inputs, ask_save_location

root = tk.Tk()

file1, file2, folder = ask_inputs(root)
output = ask_save_location(root)

print(f'Contacts File: {file1}\nDict File: {file2}\nReports Folder: {folder}')
print(f'Output: {output}')